In [4]:
import numpy as np
import pandas as pd

# Génération des données
jobs_data = {
    'Job': [1, 1, 1, 2, 2, 3, 3, 3],
    'Task': [1, 2, 3, 1, 2, 1, 2, 3],
    'Machine': ['M1', 'M2', 'M3', 'M2', 'M3', 'M1', 'M3', 'M2'],
    'Duration': [2, 3, 2, 4, 1, 3, 2, 1]
}

df_jobs = pd.DataFrame(jobs_data)
df_jobs


,Job,Task,Machine,Duration
0,1,1,M1,2
1,1,2,M2,3
2,1,3,M3,2
3,2,1,M2,4
4,2,2,M3,1
5,3,1,M1,3
6,3,2,M3,2
7,3,3,M2,1


In [3]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD

# Initialisation du modèle
model = LpProblem(name="job-shop-scheduling", sense=LpMaximize)

# Définition des variables
tasks = [(i, j) for i in range(len(df_jobs)) for j in range(i + 1, len(df_jobs))]
x = LpVariable.dicts("x", tasks, cat="Binary")
C = LpVariable.dicts("C", range(len(df_jobs)), lowBound=0, cat="Continuous")

# Ajout des contraintes
# 1. Une tâche doit être exécutée après la tâche précédente du même job
for i in range(len(df_jobs)):
    for j in range(i + 1, len(df_jobs)):
        if df_jobs.iloc[i]['Job'] == df_jobs.iloc[j]['Job']:
            model += C[j] >= C[i] + df_jobs.iloc[i]['Duration']

# 2. Une machine ne peut exécuter qu'une seule tâche à la fois
for m in df_jobs['Machine'].unique():
    for t1 in df_jobs[df_jobs['Machine'] == m].index:
        for t2 in df_jobs[df_jobs['Machine'] == m].index:
            if t1 < t2:
                model += C[t2] >= C[t1] + df_jobs.iloc[t1]['Duration'] * x[(t1, t2)]
                model += C[t1] >= C[t2] + df_jobs.iloc[t2]['Duration'] * (1 - x[(t1, t2)])

# 3. Objectif : minimiser le makespan
makespan = LpVariable("makespan", lowBound=0, cat="Continuous")
model += makespan
for i in range(len(df_jobs)):
    model += makespan >= C[i] + df_jobs.iloc[i]['Duration']

# Résolution du modèle
model.solve(PULP_CBC_CMD(msg=0))

# Résultats
print(f"Optimal Schedule Length: {makespan.value()}")
schedule = [(df_jobs.iloc[i]['Job'], df_jobs.iloc[i]['Task'], C[i].value()) for i in range(len(df_jobs))]
schedule.sort(key=lambda x: x[2])
for job in schedule:
    print(f"Job {job[0]}, Task {job[1]} starts at time {job[2]:.2f}")


Optimal Schedule Length: 20.333333
Job 1, Task 1 starts at time 0.00
Job 3, Task 1 starts at time 6.00
Job 3, Task 2 starts at time 9.00
Job 3, Task 3 starts at time 11.00
Job 1, Task 2 starts at time 12.33
Job 2, Task 1 starts at time 12.33
Job 2, Task 2 starts at time 16.33
Job 1, Task 3 starts at time 18.33


In [ ]:
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select('discrete_classification')   import geemap
m = geemap.Map()
m.set_center(-88.6, 26.4, 1)
# Ajouter la couche de couverture terrestre à la carte
m.add_layer(dataset, {
    'min': 0,
    'max': 255,
    'palette': ['white', 'green', 'blue', 'red']  # Palette pour visualiser les classes
}, 'Land Cover') # Exemple de grande ROI pour la France
roi = ee.Geometry.Polygon(
    [[
        [-5.0, 41.0],
        [10.0, 41.0],
        [10.0, 51.0],
        [-5.0, 51.0]
    ]]
)
unique_values = dataset.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=roi,
    scale=100,
    maxPixels=1e13
).get('discrete_classification')

# Afficher les valeurs uniques
print('Valeurs uniques de la bande discrete_classification :', unique_values.getInfo())  Valeurs uniques de la bande discrete_classification : {'0': 25, '100': 1885.4470588235295, '111': 11324498.968627451, '114': 23868319.05098039, '115': 9442216.588235294, '116': 5603059.803921568, '121': 372519.3176470588, '123': 1, '124': 2100910.945098039, '125': 6040851.870588236, '126': 25755471.890196085, '20': 5009925.160784313, '200': 74785960.70980404, '30': 21452704.278431375, '40': 69484469.79215689, '50': 11347766.184313728, '60': 669029.694117647, '70': 424915.4588235294, '80': 1285951.6745098038, '90': 428679.26666666666}